In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/malayalam-news/HackFake Database for the Hackathon - Sheet1.csv


In [2]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import matplotlib.pyplot as plt
tf.get_logger().setLevel('ERROR')

In [3]:
#Read dataset
FILEPATH='/kaggle/input/malayalam-news'
df_train=pd.read_csv(os.path.join(FILEPATH,'HackFake Database for the Hackathon - Sheet1.csv'))[:500]
df_train = df_train.drop(['Website (source)', 'Timeline/Date','Title'], axis=1)[2:].reset_index(drop=True)
df_train=df_train.dropna(axis=0)

In [4]:
df_train.columns

Index(['Heading of News Article', 'Text of News Article', 'Hate speech',
       'Misleading', 'Disinformation', 'Rumor/Hoax', 'Sensationalism',
       'Credible'],
      dtype='object')

In [5]:
X=df_train[['Text of News Article','Heading of News Article']]
#'Heading of News Article'
y=df_train[['Hate speech','Misleading','Disinformation','Rumor/Hoax','Sensationalism']]

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
y_train=np.array(y_train).astype('int')
y_test=np.array(y_test).astype('int')

In [6]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import regularizers,layers,Input,Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import TextVectorization,Conv1D, MaxPooling1D, LSTM, Bidirectional, Dense, BatchNormalization, Dropout

#Custom CallBack and Scheduler
early_stopping = tf.keras.callbacks.EarlyStopping(
    patience=15,
    min_delta=0.001,
    restore_best_weights=True,
)

lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
  0.001,
  decay_steps=X.shape[0]*20,
  decay_rate=1,
  staircase=False)


def tf_model():
    # Input layers for two text columns
    input_layer1 = Input(shape=(), dtype=tf.string, name='text_input1')
    input_layer2 = Input(shape=(), dtype=tf.string, name='text_input2')

    # Preprocessor layers for each text column
    preprocessor_layer = hub.KerasLayer(
        "https://tfhub.dev/google/universal-sentence-encoder-cmlm/multilingual-preprocess/2"
    )
    
    preprocessed_text1 = preprocessor_layer(input_layer1)
    preprocessed_text2 = preprocessor_layer(input_layer2)

    # Encoder layers for each text column
    encoder_layer = hub.KerasLayer(
        "https://tfhub.dev/google/LaBSE/2",
        trainable=False
    )

    embedded_text1 = encoder_layer(preprocessed_text1)["sequence_output"]
    embedded_text2 = encoder_layer(preprocessed_text2)["sequence_output"]

    # Concatenate the outputs of the two text columns
    concatenated_output = layers.Concatenate()([embedded_text1, embedded_text2])

    # Pass the concatenated embeddings through LSTM layer
    lstm_layer = tf.keras.layers.LSTM(units=64, return_sequences=False)
    lstm_output = lstm_layer(concatenated_output)

    # For example, add a Dense layer for classification
    output_layer = layers.Dense(units=5, activation='sigmoid', name='output')(lstm_output)

    # Create the model with two input layers
    model = Model(inputs=[input_layer1, input_layer2], outputs=output_layer)

    return model

# Create the model
model = tf_model()

# Display the model summary
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 text_input1 (InputLayer)    [(None,)]                    0         []                            
                                                                                                  
 text_input2 (InputLayer)    [(None,)]                    0         []                            
                                                                                                  
 keras_layer (KerasLayer)    {'input_type_ids': (None,    0         ['text_input1[0][0]',         
                             128),                                   'text_input2[0][0]']         
                              'input_word_ids': (None,                                            
                             128),                                                            

In [7]:
model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
        loss='binary_crossentropy',
        metrics=['acc']
)

In [8]:
a0,a1=X_train['Text of News Article'],X_train['Heading of News Article']
b0,b1=X_test['Text of News Article'],X_test['Heading of News Article']

In [9]:
model.fit([a0,a1],y_train,epochs=7,callbacks=[early_stopping],validation_data=[[b0,b1],y_test],batch_size=200)

Epoch 1/7
2/2 [==============================] - 169s 96s/step - loss: 0.6884 - acc: 0.1446 - val_loss: 0.6173 - val_acc: 0.2303
Epoch 2/7
2/2 [==============================] - 179s 98s/step - loss: 0.6210 - acc: 0.2380 - val_loss: 0.6036 - val_acc: 0.2364
Epoch 3/7
2/2 [==============================] - 160s 96s/step - loss: 0.5907 - acc: 0.2349 - val_loss: 0.5992 - val_acc: 0.2364
Epoch 4/7
2/2 [==============================] - 161s 97s/step - loss: 0.5670 - acc: 0.2500 - val_loss: 0.6041 - val_acc: 0.2545
Epoch 5/7
2/2 [==============================] - 189s 125s/step - loss: 0.5471 - acc: 0.2952 - val_loss: 0.6045 - val_acc: 0.2727
Epoch 6/7
2/2 [==============================] - 160s 95s/step - loss: 0.5192 - acc: 0.3072 - val_loss: 0.6096 - val_acc: 0.2788
Epoch 7/7
2/2 [==============================] - 159s 95s/step - loss: 0.4869 - acc: 0.3253 - val_loss: 0.6160 - val_acc: 0.2788
